In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug  2 18:45:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Reshape, Concatenate,ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose,UpSampling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from keras.datasets import cifar10
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm
from sklearn.utils import shuffle
import sys
import os
import numpy as np
import random
from numpy.random import randint,randn,choice,rand
from numpy import ones,zeros,vstack
import pandas as pd
import cv2
from keras.preprocessing.image import ImageDataGenerator
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import imageio
from numpy.random import randint,randn,choice,rand


Using TensorFlow backend.


In [3]:
DATA_PATH = '/content/drive/My Drive/NumpyArrayCovidx'

GENERATE_SQUARE =64
CHANNELS=1

IMAGE_SHAPE=(GENERATE_SQUARE,GENERATE_SQUARE,CHANNELS)

In [4]:
def load_data():
  training_binary_path_covid_images = os.path.join(DATA_PATH,
          f'covid19_images.npy')
  print("Loading covid training pickle...")
  covid19_images = np.load(training_binary_path_covid_images,allow_pickle=True)

  for i in range(len(covid19_images)):
    covid19_images[i] = cv2.cvtColor(covid19_images[i], cv2.COLOR_BGR2RGB)
    covid19_images[i] = cv2.resize(covid19_images[i], (GENERATE_SQUARE, GENERATE_SQUARE))

  training_data=[]
  for item in covid19_images:
    if(CHANNELS==1):
      item = np.mean(item, axis=2)
    training_data.append(item)

  train_data = np.array(training_data)
  return train_data

def load_data_fake():
  training_binary_path_covid_images = os.path.join(DATA_PATH,
          f'covid19GAN_images.npy')
  print("Loading covid training pickle...")
  covid19_images = np.load(training_binary_path_covid_images,allow_pickle=True)
  return covid19_images

In [5]:
covid19=load_data()

Loading covid training pickle...


In [6]:
covid19GAN=load_data_fake()

Loading covid training pickle...


In [7]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]


In [8]:
available_transformations = {
    'rotate': random_rotation,
    'horizontal_flip': horizontal_flip
}

In [10]:
i=0
for item in covid19:
  num_transformations_to_apply = random.randint(1, len(available_transformations))
  num_transformations = 0
  transformed_image = None
  while num_transformations <= num_transformations_to_apply:
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](item)
        num_transformations += 1
  imageio.imwrite(f"drive/My Drive/CovidxDatasetSplit/covidAugmentation/image-{i}.png", transformed_image)
  i=i+1

In [11]:
for item in covid19GAN:
  num_transformations_to_apply = random.randint(1, len(available_transformations))
  num_transformations = 0
  transformed_image = None
  while num_transformations <= num_transformations_to_apply:
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](item)
        num_transformations += 1
  imageio.imwrite(f"drive/My Drive/CovidxDatasetSplit/covidAugmentation/image-{i}.png", transformed_image)
  i=i+1